In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 프롬프트
- 입력은 딕셔너리 형태로 받음.
- 각 키는 프롬프트 내의 변수
- 변수는 나중에 실제 값으로 채워짐

### 문자열 프롬프트 템플릿
- 단일 문자열 형태의 프롬프트를 생성하는 데 사용
- 일반적으로 간단한 입력에 사용

In [2]:
from langchain_core.prompts import PromptTemplate
# 주어진 주제에 대한 조언을 프롬프트 템플릿 정의
prompt_template = PromptTemplate.from_template("주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.")
# '투자' 주제로 프롬프트 템플릿 호출
prompt_template.invoke({"topic": "투자"})

StringPromptValue(text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.')

### 챗 프롬프트 템플릿
- 대화형 AI 모델과 상호작용하는 데 필요한 메시지 시퀀스를 생성하는 구조
- 메시지 시퀀스란 대화의 흐름을 구성하는 메시지들의 연속된 집합
- 각 메시지는 시스템, 사용자, AI의 역할로 구성

In [3]:
from langchain_core.prompts import ChatPromptTemplate
# 챗 프롬프트 템플릿 정의: 사용자와 시스템 간의 메시지 포함
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic}에 대해 금융 관련 조언을 해주세요.")
])
# "주식" 주제로 챗 프롬프트 템플릿 호출
prompt_template.invoke({"topic": "주식"})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 조언을 해주세요.', additional_kwargs={}, response_metadata={})])

- 시스템 메시지(system): AI의 역할을 정의하여, AI가 어떤 종류의 응답을 제공해야 하는지를 명확히 합니다.
- 사용자 메시지(user): 사용자가 요청하는 내용을 포함하여, AI에게 특정 정보를 요청합니다.

### 메시지 자리 표시자
- 템플릿 내에서 동적으로 메시지를 삽입하는 역할
- 만약 사용자가 전달한 메시지 목록을 특정 위치에 삽입하고자 한다면 MessagePlaceholder를 사용할 수 있음

In [4]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
# (방법1) 메시지 자리 표시자를 포함한 챗 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagesPlaceholder("msgs"),
])
# 메시지 리스트를 'msgs'자리 표시자에 전달하여 호출
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [5]:
# (방법2) MessagePlaceholder 클래스를 사용하지 않고 비슷한 작업 수행
proompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("placeholder", "{msgs}"), # 여기서 msgs가 자리 표시자로 사용됩니다.
])
# 메시지 리스트를 msgs 자리 표시자에 전달하여 호출
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

## 퓨샷 프롬프트
퓨샷 프롬프트는 대규모 언어 모델이 더 나은 성능을 발휘하도록 몇가지 예제 입력과 출력을 제공하는 방식입니다.  
예제가 없으면 제로샷, 한개 있으면 원샷, n개 있으면 n샷 또는 퓨샷이라고 합니다.  

In [7]:
# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 질문과 답변을 포맷하는 프롬프트 템플릿 정의
example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

In [8]:
# 퓨샷 예제 목록 생성
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
"""
    },
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권 투자의 위험도는 어느 정도인가요?
중간 답변: 채권 투자의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
        """}
]

In [10]:
print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



결과를 보면 첫번째 예제가 프롬프트에 맞게 텍스트 형태로 변환되어 출력됩니다.

### FewShotPromptTemplate을 이용한 프롬프트 생성

In [12]:
# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate
# FewShotPromptTemplate 생성
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)

# '부동산 투자' 주제로 프롬프트 호출 및 출력
print(
    prompt.invoke({"input": "부동산 투자의 장점은 무엇인가?"}).to_string()
)

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권 투자의 위험도는 어느 정도인가요?
중간 답변: 채권 투자의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
        

질문: 부동산 투자의 장점은 무엇인가?


### 예제 선택기 사용
모든 예제를 한꺼번에 사용하지 않고, 입력된 질문가 가장 유사한 예제만 선택할 수도 있습니다.  
예제 선택기 중 SemanticSimilarityExampleSelector는 질문과 예제 사이의 유사도를 계산하여 가장 비슷한 예제를 찾아줍니다.  
이 외에도 BaseExampleSelector, LengthBasedExampleSelector등 다양한 예제 선택기가 있습니다.  
http://python.langchain.com/docs/how_to/example_selectors/

In [14]:
# 라이브러리 불러오기
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 예제 선택기 초기화
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(), # 임베딩 생성에 사용하는 함수
    Chroma, #임베딩을 저장하고 유사도 검색을 수행하는 벡터 저장소 클래스
    k=1 #선택할 예제의 수
)

langchain_chroma는 랭체인 프레임워크와 크로마 벡터 저장소를 연결하는 통합 패키지입니다.  
k=1은 입력된 질문과 가장 유사한 하나의 예제만 선택하겠다는 의미입니다.  

In [15]:
# 입력과 가장 유사한 예제 선택
question = "부동산 투자의 장점은 무엇인가?"
selected_example = example_selector.select_examples({"question": question})

In [16]:
# 선택된 예제 출력
print(f"입력 질문: {question}")
for example in selected_example:
    print("\n")
    print("# 입력과 가장 유사한 예제")
    for k, v in reversed(example.items()):
        print(f"{k}: {v}")

입력 질문: 부동산 투자의 장점은 무엇인가?


# 입력과 가장 유사한 예제
question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권 투자의 위험도는 어느 정도인가요?
중간 답변: 채권 투자의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
        


모델이 새로운 질문에 대한 답변을 생성할 때, 가장 적합한 예제를 참고하여 응답의 품질을 향상시킬 수 있습니다.  

In [17]:
# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# 예제 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="질문: {question}\n답변: {answer}",
)

- input_variables=["question", "answer"]: 이 템플릿은 두 개의 입력 변수를 사용합니다. 하나는 question이고, 다른 하나는 answer입니다.
- template="질문: {question}\n답변: {answer}": 실제로 질문과 답변을 표시하는 형식입니다. 질문과 답변이 "질문:~", "답변:~" 형식으로 나타나게 됩니다.

In [21]:
# 퓨샷 프롬프트 템플릿 설정
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 금융 관련 질문과 답변의 예입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=["input"],
)

- example_selector=example_selector: 입력된 질문과 가장 관련 있는 예제를 선택하는 예제 선택기를 지정합니다.
- example_prompt=example_prompt: 앞서 정의한 질문과 답변 형식의 프롬프트 템플릿을 사용하여 예제를 제공할 수 있도록 합니다.
- prefix="다음은 금융 관련 질문과 답변의 예입니다:": 프롬프트 앞부분에 붙는 텍스트로, AI에게 금융 관련 예제가 있음을 알립니다.
- suffix="질문: {input}\n답변:": 질문 후 AI가 답변을 생성해야 할 부분을 나타냅니다.
- input_variables=["input"]: 실제로 사용자가 입력한 질문이 들어갈 자리입니다.

In [24]:
# AI 모델 설정
model = ChatOpenAI(model_name="gpt-4o-mini")
#체인 구성 및 실행
chain = prompt | model
response = chain.invoke({"input": "부동산 투자의 장점은 무엇인가?"}) # invoke 메서드 사용
print(response.content)

부동산 투자의 장점은 다음과 같습니다:

1. **자산 가치 상승**: 시간이 지남에 따라 부동산의 가치가 상승할 가능성이 높습니다.
2. **안정적인 수입원**: 임대소득을 통해 정기적인 현금 흐름을 창출할 수 있습니다.
3. **세금 혜택**: 특정 세금 공제나 감면이 제공될 수 있어 세금 부담을 줄일 수 있습니다.
4. **인플레이션 헤지**: 대개 인플레이션이 상승할 때 부동산 가치 및 임대료도 함께 상승하여 자산 가치를 보호할 수 있습니다.
5. **포트폴리오 다변화**: 부동산을 포함함으로써 투자 포트폴리오를 다양화하여 위험을 분산시킬 수 있습니다.

후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자에 대한 단점은 무엇인가요?
중간 답변: 부동산 투자에는 높은 초기 투자 비용, 유동성 부족, 관리 비용 및 시장 변동성 등의 단점이 있습니다.

따라서 최종 답변은: 부동산 투자의 장점으로는 자산 가치 상승, 안정적인 수입원, 세금 혜택, 인플레이션 헤지, 포트폴리오 다변화가 있습니다.


# 프롬프트 허브
- https://smith.langchain.com/hub
- 프롬프트 허브 또는 랭체인 허브는 랭체인 생태계에서 프롬프트를 쉽게 공유하고 재사용할 수 있도록 지원하는 중앙저장소
- 프롬프트의 여러 버전을 관리하는 기능 제공, 특정 버전의 프롬프트를 선택하여 사용할 수 있음음

In [ ]:
from langchain import hub

# 최신 버전의 프롬프트 불러오기
prompt = hub.pull("hardkothari/prompt-maker")
# 특정 버전의 프롬프트 불러오기
prompt = hub.pull("hardkothari/prompt-maker:c5db8eee")

c:\workspace\python\rag_master\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\workspace\python\rag_master\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
